In [ ]:
from mylibraries import *
X_train = pd.read_csv('X_train_scaled.csv')
X_test = pd.read_csv('X_test_scaled.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
# Získanie názvov stĺpcov z pôvodného X_train
column_names = X_train.columns

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# Definovanie modelu DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Inicializácia modelu AdaBoost Classifier s rozhodovacím stromom ako základným odhadcom
adaboost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=7), n_estimators=50, random_state=42, algorithm='SAMME')

# Trénovanie modelu
adaboost.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_adaboost = adaboost.predict(X_test)

# Vyhodnotenie modelu
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
print(f"Presnosť modelu AdaBoost Classifier: {accuracy_adaboost:.4f}")

# Klasifikačná správa
print("Klasifikačná správa modelu AdaBoost Classifier:")
print(classification_report(y_test, y_pred_adaboost))

# Matica zámen
print("Matica zámen modelu AdaBoost Classifier:")
print(confusion_matrix(y_test, y_pred_adaboost))

# ROC krivka
y_prob_adaboost = adaboost.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob_adaboost)
plt.plot(fpr, tpr, label='ROC krivka (AUC = {:.2f})'.format(roc_auc_score(y_test, y_prob_adaboost)))
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Falošne pozitívna miera')
plt.ylabel('Pravdivo pozitívna miera')
plt.title('ROC Krivka pre AdaBoost Classifier')
plt.legend()
plt.show()

# Precision-Recall krivka
precision, recall, _ = precision_recall_curve(y_test, y_prob_adaboost)
average_precision = average_precision_score(y_test, y_prob_adaboost)
plt.plot(recall, precision, label='Precision-Recall krivka (priemer = {:.2f})'.format(average_precision))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Krivka pre AdaBoost Classifier')
plt.legend()
plt.show()

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(adaboost, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure()
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Trénovacia presnosť')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Validačná presnosť')

plt.xlabel('Počet trénovacích príkladov')
plt.ylabel('Presnosť')
plt.title('Learning Curve pre AdaBoost Classifier')
plt.legend(loc="best")
plt.show()

# Dôležitosť atribútov
feature_importances = adaboost.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]
top_n = 10
sorted_idx_top_n = sorted_idx[:top_n]

plt.barh(range(top_n), feature_importances[sorted_idx_top_n])
plt.yticks(range(top_n), column_names[sorted_idx_top_n])
plt.xlabel('Dôležitosť atribútov')
plt.title('Top {} najdôležitejších atribútov pre AdaBoost Classifier'.format(top_n))
plt.show()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Získanie dôležitosti atribútov
importances = adaboost.feature_importances_

# Zoradenie atribútov podľa ich dôležitosti
sorted_idx = np.argsort(importances)[::-1]

# Výber najdôležitejších atribútov
top_n = 10
top_features = column_names[sorted_idx][:top_n]

# Rozdelenie dát na trénovaciu a testovaciu množinu len s najdôležitejšími atribútmi
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

# Inicializácia modelu AdaBoost Classifier s rozhodovacím stromom ako základným odhadcom
adaboost_top = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=7), n_estimators=50, random_state=42)

# Trénovanie modelu na najdôležitejších atribútoch
adaboost_top.fit(X_train_top, y_train)

# Predikcia na testovacej množine
y_pred_adaboost_top = adaboost_top.predict(X_test_top)

# Vyhodnotenie modelu
accuracy_adaboost_top = accuracy_score(y_test, y_pred_adaboost_top)
print(f"Presnosť modelu AdaBoost Classifier na najdôležitejších atribútoch: {accuracy_adaboost_top:.4f}")

# Klasifikačná správa
print("Klasifikačná správa modelu AdaBoost Classifier na najdôležitejších atribútoch:")
print(classification_report(y_test, y_pred_adaboost_top))

# Matica zámen
print("Matica zámen modelu AdaBoost Classifier na najdôležitejších atribútoch:")
print(confusion_matrix(y_test, y_pred_adaboost_top))


In [ ]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Inicializácia modelu AdaBoost Classifier s rozhodovacím stromom ako základným odhadcom
adaboost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=7), n_estimators=50, random_state=42)

# Inicializácia modelu Bagging Classifier s AdaBoost ako odhadcom
bagging_adaboost = BaggingClassifier(estimator=adaboost, n_estimators=10, random_state=42)

# Trénovanie modelu Bagging s AdaBoost na trénovacích dátach
bagging_adaboost.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_bagging_adaboost = bagging_adaboost.predict(X_test)

# Vyhodnotenie modelu
accuracy_bagging_adaboost = accuracy_score(y_test, y_pred_bagging_adaboost)
print(f"Presnosť modelu Bagging s AdaBoost: {accuracy_bagging_adaboost:.4f}")

# Klasifikačná správa
print("Klasifikačná správa modelu Bagging s AdaBoost:")
print(classification_report(y_test, y_pred_bagging_adaboost))

# Matica zámen
print("Matica zámen modelu Bagging s AdaBoost:")
print(confusion_matrix(y_test, y_pred_bagging_adaboost))
